# SQL

Woooooo, I know you're all excited for more SQL work!  This week we're going to go deeper into the SQL world and start doing more filtering, aggregations, grouping, joins, and mixes of them all.  

You've already been introduced to all these topics and learned how to code them in Python.  A fair question is: why do this in SQL vs. just doing it locally?  Well, two reasons.  For one, your data might just be too darn big to bring in locally, so it's more efficient to do this on the SQL server.  The other might be that even if your data is small it might be more efficient to do the bulk of the work on the SQL server and then bring it in locally.  Doing that would mean less data transfer and ultimately faster speed.  

There are not strict rules regarding what you should do via SQL vs. Python, but I'd say it makes sense to do most of the simple selections, filtering, aggregations via SQL.  If you have to do more complicated work or a lot of exploration, it may pay to bring it in locally and work there.  But again, each situation is different so I'm not going to try and make a rulebook.  Use your best judgement.  

What are we going to learn this lesson?  

* GROUP BY - We'll first do some GROUP BY operations in order to aggregate some basic information about our movies
* HAVING - HAVING is a way to filter on the output of your GROUP BY.  For example, if you want to get the average rating for each movie, but only return those that have an average greater than 3
* JOIN - You've seen JOIN before, and now we're going to do them in SQL.  They allow us to bring together different tables into one.  
* Subqueries - Subqueries are, well, queries within queries.  You might want to get a query that meets a condition in one table, and then return only values from a different table that match that condition.  It's essentially query inception.  
* Odds and ends - We'll also learn some other random bits such as how to do math across columns, rename columns, expand our functions, make some new ones, etc.  

**Submission Instruction**

1- Replace the blank with your name (e.g. DE-Lab3_Intro_to_SQL_Sara_Riazi)

2- Run your notebook (all the outputs must be visible).

3- Download .ipynb

4- Submit on Gradescope

## Libraries and functions
We'll start by bringing in our usual libraries.  

We're also going to bring in our functions but we're going to modify them a bit to give us more info.  We're also going to make a couple new ones to check what tables are in our database and what column names are within our tables.

### Connection function
First we have install the python connector for MySQL.



In [2]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 53.5 MB/s eta 0:00:00


In [3]:
import mysql.connector
import pandas as pd

The get_conn_cur returns the connector **conn** and cursor **cur** instance that we can use to access the database.

In [4]:
mysql_address  = '131.193.32.85'
mysql_username='de_student'
mysql_password='DE_Student_PaSS'


#We are going to use a single database for all databases in this course.
#To avoid confusion, we use databasename_tablename naming convention.
mysql_database = 'my_dataengineering_dbs'
def get_conn_cur():
    cnx = mysql.connector.connect(user=mysql_username, password=mysql_password,
          host=mysql_address,
          database=mysql_database, port='3306');
    return (cnx, cnx.cursor())

In Python, when working with a MySQL database using a library like mysql.connector, the connection object (often named conn or cnx) is responsible for managing the communication between your Python code and the database server. This object handles authentication, network connections, and transactions. The cursor object (created from the connection using conn.cursor()) acts as your workspace for executing SQL queries and retrieving results. It allows you to send SQL commands to the database and fetch the results of queries. It is important to close both the cursor and the connection when you are done with them: the cursor should be closed first, once you have finished executing queries and processing results, to free up resources on both the client and server side; the connection should be closed afterwards to properly terminate the session and release any remaining resources. Closing these objects explicitly is considered best practice to prevent resource leaks, even if your script is about to end.

In [5]:
conn, cur = get_conn_cur()
conn

### Table name function

Let's make a function that allows us to quickly check what table names are present.  This is actually just the same function we used before to run a query string, but this time we're embedding just the one query string in the function so it only gets the table names.  This way you don't have to rewrite the string and then call it in the old function.

As you saw in the last lesson, our database contains something called an information_schema which contains, well, information about our database.  Within that you can get the tables, hence calling `infomation_schema.tables` and selecting just their names.  Feel free to check out the other information in `information_tables` here: https://www.postgresql.org/docs/9.1/information-schema.html!

In [6]:
def get_table_names():
    conn, cur = get_conn_cur() # get connection and cursor

    # query to get table names from my_data_engineering_dbs schema
    table_name_query = """
    SELECT table_name FROM information_schema.tables
    WHERE table_schema = 'my_dataengineering_dbs'; """

    # df = pd.read_sql(table_name_query, conn)
    cur.execute(table_name_query) # execute
    my_data = cur.fetchall() # fetch results


    # create a dataframe from the return data
    result_df = pd.DataFrame(my_data, columns=cur.column_names)

    cur.close() #close cursor
    conn.close() # close connection

    return  result_df

Now we can run just the function `get_table_names()` whenever we want to remember what tables we have.  This is a lot easier and faster than rewriting that query statement each time and running it!

In [7]:
# Run it!h
get_table_names()

,table_name
0,ticketsdb_venue
1,ticketsdb_sales
2,salesdb_customer
3,state
4,moviedb_ratings
5,salesdb_item
6,congressdb_sponsor
7,congressdb_senator
8,states_db
9,moviedb_movies


### Column names function

Let's also make a function to tell us what the column names are in a given table.  This is a touch tricker as we need to specify the table name as an argument and then insert that inside of a string.  We'll need to do just a touch of string formatting to make it work.  

`%s` as a placeholder operator that allows you to pass a specified value into a string.  You specify what value you want to place there by using `%` and then what you want to insert.  Here's a simple example:

```
>>> print('hello my name is %s' % 'dan')
hello my name is dan
```
There are easier ways to do the simple example above, but the format we just used will come in handy later when things get more complicated.  

For now, we're going to use it to allow us to sub in the name of the table we need columns names from.  

Let's make a function called `get_column_names`.  This function should take a table name as an argument.  It should then insert that name into our query string that gets the columns from the information_schema.

I'll first show you how it'll get added to our string, then we'll make the function.

In [8]:
# First, watch what happens when you assign a table name to an object and then add it to the string
tn = 'test_table_name'
column_name_query =  """SELECT column_name FROM information_schema.columns WHERE table_schema = '%s' and schema_table_name = '%s' """ % (mysql_database, tn)
column_name_query # Note how it just added the object name at the end to make a functional query

"SELECT column_name FROM information_schema.columns WHERE table_schema = 'my_dataengineering_dbs' and schema_table_name = 'test_table_name' "

In [9]:
# Just for comparisons watch what happens if we didn't use %s and instead tried to insert directly.
tn = 'test_table_name'
column_name_query =  """SELECT column_name FROM information_schema.columns WHERE table_schema = ista322dbs and table_name = tn """
column_name_query # Note how it didn't put the object 'tn' inside but just left it as a string.

'SELECT column_name FROM information_schema.columns WHERE table_schema = ista322dbs and table_name = tn '

In [10]:
# Now make a function that uses this approach.

def get_column_names(table_name): # arguement of table_name
  conn, cur = get_conn_cur() # get connection and cursor

  # Now select column names while inserting the table name into the WERE
  column_name_query =  """SELECT column_name FROM information_schema.columns
       WHERE table_name = '%s' """ %table_name

  cur.execute(column_name_query) # exectue
  my_data = cur.fetchall() # store
  result_df = pd.DataFrame(my_data, columns=cur.column_names)

  cur.close() # close
  conn.close() # close

  return result_df # return

In [11]:
# Test it out!
get_column_names(table_name = 'moviedb_movies')

,column_name
0,movieid
1,title
2,genres


Great!  These functions will be really helpful when you're joining multiple tables or doing aggregations.  They allow for you to in just one line get the info you need vs. writing a query each time.  

### Expanding our query function

Now we can create a `run_query` function for general queries using the same strategy that we used for getting column names.

Remember that after we exectue our query our cursor stores the resulting information in memory.  We use `fetchall()` on the cursor to get that info, but the cursor also contains other info such as  column names.

In [12]:
def run_query(query_string):

  conn, cur = get_conn_cur() # get connection and cursor

  cur.execute(query_string) # executing string as before

  my_data = cur.fetchall() # fetch query data as before

  result_df = pd.DataFrame(my_data, columns=cur.column_names)


  cur.close() # close
  conn.close() # close

  return result_df

Let's just test this with a quick query.  Note how the response is a tuple of two items.  The first is a list of your column names.  The second is a list of tuples of the data you requested.

In [13]:
qs = """ SELECT * FROM moviedb_ratings
            LIMIT 200; """
run_query(qs)

,userid,movieid,rating,date
0,1,296,5,2006-05-17 10:34:04
1,1,306,3,2006-05-17 07:26:57
2,1,307,5,2006-05-17 07:27:08
3,1,665,5,2006-05-17 10:13:40
4,1,899,3,2006-05-17 07:21:50
...,...,...,...,...
195,2,4535,3,2006-03-03 14:17:04
196,2,4571,1,2006-03-03 13:59:44
197,2,4720,4,2006-03-03 14:14:10
198,2,4857,1,2006-03-03 13:59:38


## GROUP BY

Let's now jump into doing some data aggregations.  For these you need two things.  First, your GROUP BY argument which tells SQL what level you want to group your data by.  Second, you need to apply some sort of aggregation function to a column.  Postgres has the following aggregation functions built in:
* AVG() - Get the average of a column within the specified group.
* COUNT() - Count how many observations are within each group.
* MAX() - Max value within a group.
* MIN() - Min value within a group.
* SUM() - Sum of values in a column within a group

Let's start by just getting the average rating for each movie.  Let's first use our `get_column_names` function to remember what we're working with.

In [14]:
get_column_names(table_name= 'moviedb_ratings')

,column_name
0,userid
1,movieid
2,rating
3,date


In [15]:
qs = """SELECT * FROM moviedb_ratings LIMIT 100"""
run_query(qs)

,userid,movieid,rating,date
0,1,296,5,2006-05-17 10:34:04
1,1,306,3,2006-05-17 07:26:57
2,1,307,5,2006-05-17 07:27:08
3,1,665,5,2006-05-17 10:13:40
4,1,899,3,2006-05-17 07:21:50
...,...,...,...,...
95,2,733,4,2006-03-03 13:58:25
96,2,858,3,2006-03-03 14:15:26
97,2,914,4,2006-03-03 14:27:22
98,2,953,4,2006-03-03 14:24:47


So, we want to get the average rating, so we'll need `AVG(rating)` in our `SELECT`.  And we want the average for each movie, which means we want to use `GROUP BY movieid`.  Let's try it.

In [16]:
# Average rating
qs = """SELECT movieid, AVG(rating) FROM moviedb_ratings
          GROUP BY movieid;"""
run_query(qs)

,movieid,AVG(rating)
0,1,3.7797
1,2,3.1255
2,3,3.0746
3,4,2.8129
4,5,2.9854
...,...,...
59042,209157,1.0000
59043,209159,3.0000
59044,209163,4.0000
59045,209169,3.0000


### Aliasing and ordering

That query worked great!  But there are some issues.  For one, notice how the column name for our average rating is just `'avg'`.  That's not very descriptive, but luckily you can rename or 'alias' columns.  You just write `as new_name` after the aggregating function.  Let's alias the column as `avg_rating`.

Similarly, the average ratings are not in any order.  Let's add an `ORDER BY` at the end to arrange by average rating. **NOTE** we need to order by the aliased column name, so in this case `avg_rating`!

In [17]:
# Average rating w/ alias and order by
qs = """SELECT movieid, AVG(rating) as avg_rating FROM moviedb_ratings
          GROUP BY movieid
          ORDER BY avg_rating;"""
run_query(qs)

,movieid,avg_rating
0,191845,0.0000
1,177417,0.0000
2,185055,0.0000
3,208008,0.0000
4,195183,0.0000
...,...,...
59042,124174,5.0000
59043,159471,5.0000
59044,159904,5.0000
59045,187949,5.0000


**Question 1**  Find the count of how many times each movie has been reviewed?  Alias the value as `number_reviews` and sort by that value.

In [18]:
qs = """SELECT movieid, COUNT(rating) as number_reviews FROM moviedb_ratings
          GROUP BY movieid
          ORDER BY number_reviews;
          """
run_query(qs)

,movieid,number_reviews
0,94308,1
1,194961,1
2,84886,1
3,161000,1
4,190045,1
...,...,...
59042,2571,72674
59043,593,74127
59044,296,79672
59045,318,81482


### Grouping by time

Just like in python, you can group by time values.  There are a couple ways to do this, but here we're going to use a built in function called `year()`. This function get the year part from the timestamp. We have similar functions for day, month, etc.

In this case, our code will be `SELECT year(date) as year, ....`.  This is saying "grab the year part from the datetime that's inside the timestamp column and alias it as 'year'".  

In [19]:
# review average per year
qs = """ SELECT year(date) as year, AVG(rating) FROM moviedb_ratings
          GROUP BY year
          ORDER BY year
          """
run_query(qs)

,year,AVG(rating)
0,1995,3.6667
1,1996,3.5457
2,1997,3.5878
3,1998,3.5127
4,1999,3.6167
5,2000,3.5751
6,2001,3.5324
7,2002,3.4852
8,2003,3.3315
9,2004,3.2092


## HAVING

HAVING is just essentially like WHERE but instead works on your grouped results.  For example, let's get just years where the average ratings are greater than 3.5.  

The only place it differs is that you call your aggregation again for the HAVING statement.  For example, we'd need `HAVING AVG(rating) >= 3.5` to filter only years that have an average >= 4.  Using the column alias we created in the select wouldn't work (e.g. `HAVING avg_rating >= 3.5`) due to order of SQL operations... more on that later.


In [20]:
# Average per year where avg >= 3.5
qs = """ SELECT year(date) as year, AVG(rating) as avg_rating FROM moviedb_ratings
          GROUP BY year
          HAVING AVG(rating) >= 3.5
          """
run_query(qs)

,year,avg_rating
0,1995,3.6667
1,1996,3.5457
2,1997,3.5878
3,1998,3.5127
4,1999,3.6167
5,2000,3.5751
6,2001,3.5324


**Question 2**  Get a count of how many times a movie has been reviewed.  But this time display only the movieid's and counts of movies that have been reviewed more than 50 times.

In [21]:
qs = """SELECT movieid, COUNT(rating) as number_reviews FROM moviedb_ratings
          GROUP BY movieid
          HAVING COUNT(rating) > 50
          """
run_query(qs)

,movieid,number_reviews
0,1,57309
1,2,24228
2,3,11804
3,4,2523
4,5,11714
...,...,...
13082,205076,53
13083,205383,252
13084,205425,69
13085,206499,90


## JOIN

You've already done joins in python using `.merge()`, but now we're going to do them straight away in our database.  The concept is the exactly same in that there are different types of joins, LEFT being the most common, that are connected by a key.  

To do a JOIN you specify the type (LEFT JOIN, INNER JOIN, OUTER JOIN), then the dataframe you want to join, then ON followed by the columns to join on.  

For example, below let's join movie sales data to our dataset of movie info.  

In [22]:
# A quick look at the head of movies again
qs = """SELECT * FROM moviedb_movies
          LIMIT 5;"""
run_query(qs)

,movieid,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [23]:
# A quick look at the head of movie_sales
qs = """SELECT * FROM moviedb_movie_sales
          LIMIT 5;"""
run_query(qs)

,movieid,domestic,international
0,0,8222319,17008452
1,1,87368899,54052102
2,2,10051688,4572996
3,3,86658853,77643014
4,4,84781543,65428731


We want to join the sales data *to* our movie data, so the FROM will be `FROM movies` and we'll LEFT JOIN movie_sales using, well ,`LEFT JOIN movie_sales`.

Note in order to tell it what columns you have your keys in you need to use the format `table_name.column_with_key`.  This is because we have the same column name, `movieid`, in both datasets so it needs to know which one to use from each.

In [24]:
# Join sales data to movies data
qs = """ SELECT * FROM moviedb_movies
          LEFT JOIN moviedb_movie_sales ON moviedb_movies.movieid = moviedb_movie_sales.movieid
          LIMIT 3;
          """
run_query(qs)

,movieid,title,genres,movieid,domestic,international
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,87368899,54052102
1,2,Jumanji (1995),Adventure|Children|Fantasy,2,10051688,4572996
2,3,Grumpier Old Men (1995),Comedy|Romance,3,86658853,77643014


That worked!  Messy formatting aside, you can see that now we have a single return that contains the movie ID, title, genres, as well as domestic and international gross in sales.

What if you wanted to attach the movie title to each individual review?  LEFT JOIN will add that info from `movies` to every row of `ratings`.

In [25]:
# Join sales data to movies data
qs = """ SELECT * FROM moviedb_ratings
          LEFT JOIN moviedb_movies ON moviedb_ratings.movieid = moviedb_movies.movieid
          LIMIT 2;
          """
run_query(qs)

,userid,movieid,rating,date,movieid,title,genres
0,1,296,5,2006-05-17 10:34:04,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3,2006-05-17 07:26:57,306,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama


### Selecting specific columns during a join

Here's another order of operations thing...  even though it looks like your data isn't joined until later, that's not strictly true based on order of operations.  What this means is that you can actually select columns from both tables in your SELECT statement.  This will let you keep only what you want from both tables.  

Let's run the same query as above but only get `userid, movieid, rating, date, and title`. Note that just like we have to use the `table_name.column_name` format in our ON, we also need to use that in our SELECT to get just one of the `movieid` columns.

In [26]:
# Join movie data to ratings data
qs = """ SELECT userid, moviedb_ratings.movieid, rating, date, title FROM moviedb_ratings
          LEFT JOIN moviedb_movies ON moviedb_ratings.movieid = moviedb_movies.movieid
          LIMIT 2;
          """
run_query(qs)

,userid,movieid,rating,date,title
0,1,296,5,2006-05-17 10:34:04,Pulp Fiction (1994)
1,1,306,3,2006-05-17 07:26:57,Three Colors: Red (Trois couleurs: Rouge) (1994)


## Subqueries

In python you would commonly filter results by several elements in different columns.  For example, if you had a bunch of spotify data in a single table you might filter by artist name and tempo and number of listens.  It makes sense to do the same thing here in SQL land, but it's a touch trickier as often those data will be in different tables.  So, it makes it difficult if you want to retrieve data from one table that meets a condition that's in another table.  

For example, what if we want to get only the movie ratings for movies that are a comedy.  Ratings are in one table, `ratings`, but we want to filter by genre which is over in the `movies` table.  To do this we need to use a subquery.  Essentially we'll filter via WHERE `ratings` by the output of another query on `movies`.

Let me show you the whole thing and then I'll break it down a bit more.  

In [27]:
qs = """ SELECT * FROM moviedb_ratings
            WHERE movieid IN (SELECT movieid FROM moviedb_movies WHERE genres LIKE '%Comedy%')
            LIMIT 20;"""
run_query(qs)

,userid,movieid,rating,date
0,1,296,5,2006-05-17 10:34:04
1,1,665,5,2006-05-17 10:13:40
2,1,899,3,2006-05-17 07:21:50
3,1,1175,3,2006-05-17 07:27:06
4,1,2011,2,2006-05-17 07:14:39
5,1,2012,2,2006-05-17 07:14:28
6,1,2843,4,2006-05-17 07:28:11
7,1,3448,4,2006-05-17 07:21:20
8,1,3569,5,2006-05-17 10:26:43
9,1,4973,4,2006-05-17 07:31:20


In [28]:
qs = """ SELECT movieid, rating, year(date) as year FROM moviedb_ratings
            WHERE movieid IN (SELECT movieid FROM moviedb_movies WHERE genres LIKE '%Comedy%')
            LIMIT 20;"""
run_query(qs)

,movieid,rating,year
0,296,5,2006
1,665,5,2006
2,899,3,2006
3,1175,3,2006
4,2011,2,2006
5,2012,2,2006
6,2843,4,2006
7,3448,4,2006
8,3569,5,2006
9,4973,4,2006


I limited the return so it wasn't overly long. But, you can see that only a few of the movieid's are represented.  In this case, only those that are comedies.  

Let's take a second and run the subquery to show you what it would return.

In [29]:
qs = """ SELECT movieid FROM moviedb_movies
            WHERE genres LIKE '%Comedy%' """
run_query(qs)

,movieid
0,1
1,3
2,4
3,5
4,7
...,...
16865,209119
16866,209141
16867,209153
16868,209155


See, it gives us just those ID's where 'Comedy' appears in the genres column.  You could in theory hardcode this without the subquery.  For example, you could have used the following:

```
qs = """ SELECT * FROM ratings
            WHERE movieid IN (1, 3, 4, 5, 7);"""
```

The obvious problem here is that this isn't dynamic.  What if you want a different genre?  You'd have to go and first do a query in `movies` to find out all the ID's that match, then hardcode them in this second query.  And if your data changes you'd have to update it.  Or, if your data are even remotely long it's just not possible to hardcode.  Remember that this full dataset has 67k movies, of which likely 1/4 are comedies.  You can't reasonably hardcode that so you *must* use a subquery!

**Question 3**  Write a query that gets the titles of movies that made over 99 million dollars in domestic sales? (the numbers of not real, so don't get surprised :) )


In [30]:
qs = """SELECT title as year FROM moviedb_movies
            WHERE movieid IN (SELECT movieid FROM moviedb_movie_sales WHERE domestic > 99000000);"""
run_query(qs)

,year
0,Assassins (1995)
1,Vampire in Brooklyn (1995)
2,If Lucy Fell (1996)
3,Gospa (1995)
4,Houseguest (1994)
...,...
123,Indiana Jones and the Kingdom of the Crystal S...
124,Shutter (2008)
125,"Klansman, The (1974)"
126,Variety Lights (1950)


Of course, you can have a more complicated subquery.  For example what if we want to get just the titles of movies where the average rating is 3.5 or greater.  We'll need to do a GROUP BY in our subquery.  

In [37]:
qs = """ SELECT title FROM moviedb_movies
            WHERE movieid IN (SELECT movieid FROM moviedb_ratings
                                GROUP By movieid
                                HAVING AVG(rating) >= 3.5) """
run_query(qs)

,title
0,Toy Story (1995)
1,Heat (1995)
2,"American President, The (1995)"
3,Casino (1995)
4,Sense and Sensibility (1995)
...,...
10817,Adrenalin: The BMW Touring Car Story (2014)
10818,Square Roots: The Story of SpongeBob SquarePan...
10819,Destination Titan (2011)
10820,Santosh Subramaniam (2008)


# Feature generation

The last thing I'm going to show you is just that you can do more manipulations of columns in your select.  Specifically, you can do math with columns in SELECT. Obviously you can do this rapidly in python, but why bring in a bunch of columns to do simple math on when you can bring in just the resulting one.  

For example, below I get the total_sales by adding the domestic and international columns. I also alias it as 'total_sales'.  

In [ ]:
# Add two columns plus an alias
qs = """ SELECT movieid, (domestic+international) as total_sales FROM moviedb_movie_sales;
          """
run_query(qs)